In [2]:
from llama_cpp import Llama

In [12]:
llm = Llama(model_path="/data/mistral/query-to-mql/exp-10/nov-20/merged-model/ggml-model-q4_0/ggml-model-q4_0.gguf", n_threads=8)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 1 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [13]:
query_template_v1 = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""

context = """{
    "MEASURE": [{"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]}],
    "FILTER": [{"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date"]}]
    }"""

date_input = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [14]:
user_query = 'sales in nov 2020 to may 2021 vs Q3 2021'

inp = query_template_v1.format(context=context,
                                   user_query=user_query,
                                  date_input=date_input)

In [16]:
import time

In [22]:
start = time.time()
output = llm(
  inp, # Prompt
  max_tokens=200, # Generate up to 32 tokens
  echo=True, # Echo the prompt back in the output
    temperature=0
)
print("time taken : ", time.time()-start)

SyntaxError: keyword argument repeated (<ipython-input-22-c7531f2f955e>, line 6)

In [19]:
print(output)

{'id': 'cmpl-89fcd03f-1524-414c-bc5d-738edb0ed24b', 'object': 'text_completion', 'created': 1700931069, 'model': '/data/mistral/query-to-mql/exp-10/nov-20/merged-model/ggml-model-q4_0/ggml-model-q4_0.gguf', 'choices': [{'text': 'Given the context : {\n    "MEASURE": [{"ENTITY": "Sales", "other names": ["sales", "sale"]}],\n    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]}],\n    "FILTER": [{"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],\n    "DERIVED MEASURE": [],\n    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date"]}]\n    } and date reference: {\'start_date\': \'01/01/2020\', \'end_date\': \'15/09/2023\'}, the query: sales in nov 2020 to may 2021 vs Q3 2021, is converted into below shown structured output.\n[MQL]\n{\'DATE VARIABLE\': {\'nov 2020 to may 2021\': [{\'CONVERTED TIME ELEMENT\': \'nov 2020 to may 2021\', \'DATE RANGE\': \'2020/11/01 - 2021/05/31\', \'ENTITY\': 

In [21]:
output['choices'][0]['text'].split('[MQL]')[]

'Given the context : {\n    "MEASURE": [{"ENTITY": "Sales", "other names": ["sales", "sale"]}],\n    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]}],\n    "FILTER": [{"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],\n    "DERIVED MEASURE": [],\n    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date"]}]\n    } and date reference: {\'start_date\': \'01/01/2020\', \'end_date\': \'15/09/2023\'}, the query: sales in nov 2020 to may 2021 vs Q3 2021, is converted into below shown structured output.\n[MQL]\n{\'DATE VARIABLE\': {\'nov 2020 to may 2021\': [{\'CONVERTED TIME ELEMENT\': \'nov 2020 to may 2021\', \'DATE RANGE\': \'2020/11/01 - 2021/05/31\', \'ENTITY\': \'Order Date\'}]}, \'MEASURE\': {\'sales\': [{\'ENTITY\': \'Sales\'}]}}\n[/MQL]\nthe steps and rationale used to achieve above structured output is as below.\nStep 1: Identify the components in the query\n- The query mentions "s